In [2]:
# !pip install pyomo
# !pip install --upgrade --user ortools
# !apt-get install -y glpk-utils
# !sudo apt-get update -y
# !sudo apt-get install -y coinor-cbc

In [3]:
# !pip install -U git+https://github.com/coin-or/pulp
# !sudo pulptest
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import pandas as pd
from pulp import *
full_player_scores = pd.read_csv('full_players.csv', index_col=0)
full_player_scores = full_player_scores.reset_index(drop=True)
full_player_scores['Score'] = pd.to_numeric(full_player_scores['Score'], errors='coerce')
# full_player_scores['Value'] = full_player_scores['Score']/full_player_scores['Price']

full_player_scores_2021 = full_player_scores[full_player_scores['Year']==2021]
full_player_scores = full_player_scores[full_player_scores['Year']==2020]

# Only keep players who are playing this season
full_player_scores[full_player_scores['Name'].isin(full_player_scores_2021['Name'].unique())].dropna(how='all').dropna(how='all')


# full_player_scores = full_player_scores[full_player_scores.isin(full_player_scores_2021['Name'].unique())]['Name'].dropna(how='all')

player_num_games_played = full_player_scores[full_player_scores['Score']>0].groupby('Name')['Team 2'].transform('count')

opponent_average_score = full_player_scores.groupby('Team 2')['Score'].transform('mean')
player_average_score = full_player_scores.groupby('Name')['Score'].transform('mean')
player_average_value = player_average_score/full_player_scores['Price']

na_locs = full_player_scores['Score'].isna()

full_player_scores.loc[na_locs,'Score'] = ((opponent_average_score+player_average_score)/2).loc[na_locs]


RU_Players = full_player_scores[(full_player_scores['Year']==2020)]

RU_Players = RU_Players.sort_values('Score',ascending=False)
RU_Players['id'] = "Round:"+RU_Players['Round'].astype(str)+"_Player:"+RU_Players['Name']


# Only take players who played more than 5 games
print(len(RU_Players['Name'].unique()))
players_to_keep = player_num_games_played[player_num_games_played>5].index.values
RU_Players = RU_Players.reindex(index=players_to_keep).dropna(how='all')
print(len(RU_Players['Name'].unique()))


# Only take players in top quater of average values (score/price)
player_best_values = player_average_value.sort_values(ascending=False)[:int(len(player_average_value)/2)]
RU_Players = RU_Players.reindex(index=player_best_values.index.values).dropna(how='all') 
print(len(RU_Players['Name'].unique()))

RU_Players = RU_Players[RU_Players['Round']<4]
# RU_Players = RU_Players.head(40)
RU_Players.groupby('Position').count()['Name']

652
501
212


Position
DE    155
FO    147
MI    122
RU     22
Name: Name, dtype: int64

In [10]:
trades_allowed = None
player_contraints = {}
prob = LpProblem("aflProblem", LpMaximize)
overall_score = LpVariable('OverallScore',0)
for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems()):
  p_id = p_id[1]
  player_contraints[p_id] = LpVariable(p_id, 0, 1, cat='Binary')

player_contraints = LpVariable.dicts("player_contraints", player_contraints, 0, 1, cat='Binary')
prob += lpSum([player_contraints[p_id[1]]*score[1] for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems())]), "Total score is maximized"

# START TRANFERS
RU_Players_prev_round = RU_Players
RU_Players_prev_round['Round_prev'] = RU_Players_prev_round['Round'] - 1

RU_transfers = RU_Players[['id','Position','Round','Name']].merge(RU_Players_prev_round[['id','Position','Round_prev','Name']],left_on=['Position','Round'],right_on=['Position','Round_prev'], suffixes = ('_prev','_next'))
RU_transfers['Transfer'] = RU_transfers['id_prev']+'->'+RU_transfers['id_next']

transfer_contraints = {}

# Define transfer itermediaries
for i,t_id in RU_transfers['Transfer'].iteritems():
  transfer_contraints[t_id] = LpVariable(t_id, 0, 1, cat='Binary')

transfer_contraints = LpVariable.dicts("transfer_contraints", transfer_contraints, 0, 1, cat='Binary')

# map rounds end to itermediary
for prev_player,trans in RU_transfers.groupby(['id_prev'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[prev_player], "Previous player equals transfer intermediatary for "+prev_player

# map intermediary to next round
for next_player,trans in RU_transfers.groupby(['id_next'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[next_player], "Next player equals transfer intermediatary for "+next_player

for round, transfers in RU_transfers[RU_transfers['Name_prev'] != RU_transfers['Name_next']].groupby(['Round'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in transfers]) <= 4, f"Round: {round}, has less than orequal to 4 transfers"
# END TRANFERS




# START total players 
for r, player in RU_Players[['id','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
  prob += lpSum([player_contraints[p_id] for p_id in player]) == 30, f"Must have 30 players in round {r}"
# END total players 




# START money contraint
money = 13 * 10**6
for round,player in RU_Players[['id','Price','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
  prob += lpSum([player_contraints[p_id]*RU_Players[RU_Players['id']==p_id]['Price'].values[0] for p_id in player]) <= money, f"Round: {round}, has less than ${money}"
# END money contraint

#START Contrain each player to a single position per round
RU_Players_Position_Casting = RU_Players.copy()
RU_Players_Position_Casting['pl_pos_id'] = (RU_Players['id']+'_'+RU_Players['Position'])

Players_Position_Casting_Series = RU_Players_Position_Casting[['id','pl_pos_id','Round']].drop_duplicates().groupby(['Round','id' ])['pl_pos_id'].apply(list).iteritems()

Players_Position_Casting = {}
for i,r in RU_Players_Position_Casting.iterrows():
  Players_Position_Casting[r['pl_pos_id']] = LpVariable(r['pl_pos_id'], 0, 1, cat='Binary')

for (round, p_id), pl_pos_ids in Players_Position_Casting_Series:
    #   Players_Position_Casting[p_id] = [LpVariable(p_id, 0, 1, cat='Binary' for p_id  in pl_pos_id)] 
    player_possible_positions = [Players_Position_Casting[pl_pos_id] for pl_pos_id  in pl_pos_ids] 
    # print(p_id)
    prob += lpSum(player_possible_positions) == player_contraints[p_id], f"Must have only have equal to {p_id} (0 or 1) of {player_possible_positions} positions in round {round}"
#END Contrain each player to a single position per round

# START max players from each position
allowed_holds_per_position = {'DE': 8, "MI" : 10, 'RU' : 3, 'FO':9}

for (position,round), pl_pos_ids in RU_Players_Position_Casting[['pl_pos_id','Position','Round']].drop_duplicates().groupby(['Position', 'Round'])['pl_pos_id'].apply(list).iteritems():
  prob += lpSum([player_contraints[p_id] for pl_pos_id in pl_pos_ids]) >= allowed_holds_per_position[position], f"Position: {position}, has less than {allowed_holds_per_position[position]} in round {round}"

# END max players from each position


print('done with pre-work')


done with pre-work


In [11]:
# prob.solve(pulp.PULP_CBC_CMD(msg=True, maxSeconds=200))

solver = getSolver('COIN_CMD', maxSeconds=200, msg=True,gapRel = 0.15, threads=100)
prob.solve(solver)


1

In [67]:
results = []
for player_position in Players_Position_Casting.values():
    if player_position.value() != 0:
        results.append([player_position.name,1])
Players_selected = pd.DataFrame(results, columns=['pl_pos_id', 'is_selected'])
RU_Players_Position_Casting['pl_pos_id'] = RU_Players_Position_Casting['pl_pos_id'].str.replace(' ', '_')
RU_Players_Position_Casting = RU_Players_Position_Casting.merge(Players_selected, on = ['pl_pos_id'])
RU_Players_Position_Casting.to_csv('solution.csv')

In [6]:
roundplayers = {1: ['Rory Lobb', 'Ben McEvoy', 'Jordan Clark', 'Adam Tomlinson', 'Harry Taylor', 'Brandon Starcevich', 'Orazio Fantasia', 'Connor Budarick', 'Daniel Talia', 'Phil Davis', 'Benjamin Stratton', 'James Frawley', 'Fischer McAsey', 'Ryan Gardner', 'Michael Walters', 'Kyle Langford', 'Shai Bolton', 'Isaac Smith', 'Tom Phillips', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Jarrod Brander', 'Nick Blakey', 'Tom Scully', 'Steven Motlop', 'Ben King', 'Paddy Dow', 'Jonathon Patton', 'Chayce Jones'], 2: ['Rory Lobb', 'Ben McEvoy', 'Adam Tomlinson', 'Harry Taylor', 'Brandon Starcevich', 'Connor Budarick', 'Daniel Talia', 'Phil Davis', 'Benjamin Stratton', 'James Frawley', 'Fischer McAsey', 'Ryan Gardner', 'Michael Walters', 'Kyle Langford', 'Isaac Smith', 'Tom Phillips', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Jye Caldwell', 'Jarrod Brander', 'Lin Jong', 'Nick Blakey', 'Tom Scully', 'Laitham Vandermeer', 'Steven Motlop', 'Ed Richards', 'Ben King', 'Jonathon Patton', 'Chayce Jones'], 3: ['Rory Lobb', 'Ben McEvoy', 'Esava Ratugolea', 'Brodie Smith', 'Harry Taylor', 'Brandon Starcevich', 'Paul Ahern', 'Connor Budarick', 'Daniel Talia', 'Phil Davis', 'Benjamin Stratton', 'Will Hamill', 'James Frawley', 'Fischer McAsey', 'Michael Walters', 'Isaac Smith', 'Tom Phillips', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Jye Caldwell', 'Jarrod Brander', 'Lin Jong', 'Nick Blakey', 'Tom Scully', 'Laitham Vandermeer', 'Ed Richards', 'Ben King', 'Jonathon Patton', 'Chayce Jones'], 4: ['Rory Lobb', 'Ben McEvoy', 'Esava Ratugolea', 'Brodie Smith', 'Brandon Starcevich', 'Orazio Fantasia', 'Paul Ahern', 'Connor Budarick', 'Daniel Talia', 'Phil Davis', 'Benjamin Stratton', 'Will Hamill', 'James Frawley', 'Fischer McAsey', 'Joel Smith', 'Michael Walters', 'Isaac Smith', 'Tom Phillips', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Josh Battle', 'Jarrod Brander', 'Nick Blakey', 'Tom Scully', 'Laitham Vandermeer', 'Kane Farrell', 'Ed Richards', 'Ben King', 'Chayce Jones'], 5: ['Rory Lobb', 'Ben McEvoy', 'Esava Ratugolea', 'Mabior Chol', 'Brodie Smith', 'Mark Blicavs', 'Harry Taylor', 'Orazio Fantasia', 'Connor Budarick', 'Daniel Talia', 'Phil Davis', 'Benjamin Stratton', 'Will Hamill', 'James Frawley', 'Joel Smith', 'Michael Walters', 'Isaac Smith', 'Tom Phillips', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Josh Battle', 'Nick Blakey', 'Tom Scully', 'Laitham Vandermeer', 'Kane Farrell', 'Ed Richards', 'Ben King', 'Jack Mahony', 'Chayce Jones'], 6: ['Rory Lobb', 'Ben McEvoy', 'Esava Ratugolea', 'Mabior Chol', 'Brodie Smith', 'Mark Blicavs', 'Harry Taylor', 'Orazio Fantasia', 'Connor Budarick', 'Daniel Talia', 'Benjamin Stratton', 'Will Hamill', 'James Frawley', 'Fischer McAsey', 'Michael Walters', 'Isaac Smith', 'Tom Phillips', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Josh Battle', 'Nick Blakey', 'Laitham Vandermeer', 'Kane Farrell', 'Ed Richards', 'Sam Weideman', 'Brayden Ainsworth', 'Jack Mahony', 'Chayce Jones'], 7: ['Peter Ladhams', 'Rory Lobb', 'Ben McEvoy', 'Esava Ratugolea', 'Mabior Chol', 'Mark Blicavs', 'Harry Taylor', 'Connor Budarick', 'Daniel Talia', 'Benjamin Stratton', 'Will Hamill', 'James Frawley', 'Fischer McAsey', 'Dustin Martin', 'Michael Walters', 'Tom Phillips', 'Zac Bailey', 'Sam Powell-Pepper', 'Brett Bewley', 'Justin Westhoff', 'Nick Blakey', 'Laitham Vandermeer', 'Kane Farrell', 'Ed Richards', 'Sam Weideman', 'Brayden Ainsworth', 'Jack Mahony', 'Elijah Taylor'], 8: ['Peter Ladhams', 'Ben McEvoy', 'Esava Ratugolea', 'Mabior Chol', 'Mark Blicavs', 'Jake Lever', 'Harry Taylor', 'Ben McKay', 'Connor Budarick', 'Daniel Talia', 'Will Hamill', 'James Frawley', 'Fischer McAsey', 'Dustin Martin', 'Michael Walters', 'Shai Bolton', 'Tom Phillips', 'Zac Bailey', 'Sam Powell-Pepper', 'Justin Westhoff', 'Bailey Scott', 'Nick Blakey', 'Laitham Vandermeer', 'Kane Farrell', 'Jake Melksham', 'Ed Richards', 'Sam Weideman', 'Brayden Ainsworth', 'Jack Mahony', 'Elijah Taylor'], 9: ['Peter Ladhams', 'Ben McEvoy', 'Esava Ratugolea', 'Mabior Chol', 'Mark Blicavs', 'Jake Lever', 'Harry Taylor', 'Ben McKay', 'Connor Budarick', 'Daniel Talia', 'Aiden Bonar', 'Will Hamill', 'Dustin Martin', 'Shai Bolton', 'Tom Phillips', 'Zac Bailey', 'Sam Powell-Pepper', 'Justin Westhoff', 'Josh Battle', 'Bailey Scott', 'Laitham Vandermeer', 'Kane Farrell', 'Jake Melksham', 'Josh Thomas', 'Ed Richards', 'James Stewart', 'Sam Weideman', 'Brayden Ainsworth', 'Jack Mahony', 'Elijah Taylor'], 10: ['Peter Ladhams', 'Mabior Chol', 'Mark Blicavs', 'Adam Tomlinson', 'Matt Guelfi', 'Jake Lever', 'Harry Taylor', 'Ben McKay', 'Connor Budarick', 'Daniel Talia', 'Aiden Bonar', 'Will Hamill', 'Fischer McAsey', 'Dustin Martin', 'Shai Bolton', 'Tom Phillips', 'Zac Bailey', 'Sam Powell-Pepper', 'Justin Westhoff', 'Josh Battle', 'Bailey Scott', 'Anthony McDonald-Tipungwuti', 'Laitham Vandermeer', 'Kane Farrell', 'Jake Melksham', 'Josh Thomas', 'Ed Richards', 'James Stewart', 'Sam Weideman', 'Jack Mahony'], 11: ['Peter Ladhams', 'Mabior Chol', 'Mark Blicavs', 'Adam Tomlinson', 'Matt Guelfi', 'Jake Lever', 'Harry Taylor', 'Martin Gleeson', 'Liam Jones', 'Ben McKay', 'Connor Budarick', 'Aiden Bonar', 'Fischer McAsey', 'Shai Bolton', 'Tom Phillips', 'Zac Bailey', 'Sam Powell-Pepper', 'Josh Battle', 'Bailey Scott', 'Anthony McDonald-Tipungwuti', 'Laitham Vandermeer', 'Kane Farrell', 'Jake Melksham', 'Dylan Clarke', 'Josh Thomas', 'Ed Richards', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Jack Mahony'], 12: ['Ben McEvoy', 'Mabior Chol', 'Mark Blicavs', 'Adam Tomlinson', 'Matt Guelfi', 'Jake Lever', 'Harry Taylor', 'Martin Gleeson', 'Liam Jones', 'Ben McKay', 'Lewis Melican', 'Connor Budarick', 'Fischer McAsey', 'Shai Bolton', 'Tom Phillips', 'Sam Powell-Pepper', 'Jack Newnes', 'Josh Battle', 'Bailey Scott', 'Nick Blakey', 'Anthony McDonald-Tipungwuti', 'Kane Farrell', 'Jake Melksham', 'Dylan Clarke', 'Josh Thomas', 'Ed Richards', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Jack Mahony'], 13: ['Ben McEvoy', 'Mabior Chol', 'Mark Blicavs', 'Adam Tomlinson', 'Matt Guelfi', 'Jake Lever', 'Daniel Howe', 'Martin Gleeson', 'Liam Jones', 'Ben McKay', 'Lewis Melican', 'Connor Budarick', 'Lachlan Keeffe', 'Shai Bolton', 'Tom Phillips', 'Sam Powell-Pepper', 'Jack Newnes', 'Josh Battle', 'Bailey Scott', 'Nick Blakey', 'Anthony McDonald-Tipungwuti', 'Tom Scully', 'Jake Melksham', 'Dylan Clarke', 'Josh Thomas', 'Ed Richards', "Tim O'Brien", 'James Stewart', 'Sam Weideman', "Xavier O'Neill"], 14: ['Ben McEvoy', 'Adam Tomlinson', 'Matt Guelfi', 'Jake Lever', 'Daniel Howe', 'Martin Gleeson', 'Liam Jones', 'Lewis Melican', 'Connor Budarick', 'Lachlan Keeffe', 'Brandon Zerk-Thatcher', 'Kyle Langford', 'Shai Bolton', 'Tom Phillips', 'Sam Powell-Pepper', 'Jack Newnes', 'Bailey Scott', 'Nick Blakey', 'Anthony McDonald-Tipungwuti', 'Tom Scully', 'Jake Melksham', 'Dylan Clarke', 'Josh Thomas', 'Ed Richards', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Tom McCartin', "Xavier O'Neill", 'Mark Hutchings'], 15: ['Ben McEvoy', 'Rory Laird', 'Adam Tomlinson', 'Lachlan Sholl', 'Matt Guelfi', 'Jake Lever', 'Daniel Howe', 'Martin Gleeson', 'Liam Jones', 'Lewis Melican', 'Lachlan Keeffe', 'Brandon Zerk-Thatcher', 'Joel Smith', 'Kyle Langford', 'Shai Bolton', 'Tom Phillips', 'Jack Newnes', 'Keidean Coleman', 'Nick Blakey', 'Anthony McDonald-Tipungwuti', 'Tom Scully', 'Jake Melksham', 'Dylan Clarke', 'Josh Thomas', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Tom McCartin', "Xavier O'Neill", 'Mark Hutchings'], 16: ['Rory Laird', 'Mark Blicavs', 'Adam Tomlinson', 'Lachlan Sholl', 'Matt Guelfi', 'Jake Lever', 'Daniel Howe', 'Martin Gleeson', 'Liam Jones', 'Lewis Melican', 'Lachlan Keeffe', 'Brandon Zerk-Thatcher', 'Joel Smith', 'Patrick Dangerfield', 'Kyle Langford', 'Jack Newnes', 'Keidean Coleman', 'Nick Blakey', 'Anthony McDonald-Tipungwuti', 'Tom Scully', 'Jake Melksham', 'Dylan Clarke', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Tom McCartin', "Xavier O'Neill", 'Roarke Smith', 'Brayden Ainsworth', 'Mark Hutchings'], 17: ['Esava Ratugolea', 'Mark Blicavs', 'Adam Tomlinson', 'Lachlan Sholl', 'Matt Guelfi', 'Jake Lever', 'Daniel Howe', 'Martin Gleeson', 'Liam Jones', 'Lewis Melican', 'Lachlan Keeffe', 'Brandon Zerk-Thatcher', 'Joel Smith', 'Patrick Dangerfield', 'Dustin Martin', 'Jack Newnes', 'Keidean Coleman', 'Jack Riewoldt', 'Nick Blakey', 'Jake Melksham', 'Dylan Clarke', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Tom McCartin', "Xavier O'Neill", 'Roarke Smith', 'Brayden Ainsworth'], 18: ['Esava Ratugolea', 'Mabior Chol', 'Mark Blicavs', 'Adam Tomlinson', 'Lachlan Sholl', 'Jake Kolodjashnij', 'Matt Guelfi', 'Jake Lever', 'Jed Bews', 'Martin Gleeson', 'Liam Jones', 'Lewis Melican', 'Lachlan Keeffe', 'Brandon Zerk-Thatcher', 'Joel Smith', 'Patrick Dangerfield', 'Dustin Martin', 'Jack Steven', 'Keidean Coleman', 'Jack Riewoldt', 'Nick Blakey', 'Jake Melksham', 'Dylan Clarke', "Tim O'Brien", 'James Stewart', 'Sam Weideman', 'Tom McCartin', 'Roarke Smith', 'Brayden Ainsworth'], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: []}


In [8]:
roundplayers[1]

['Rory Lobb',
 'Ben McEvoy',
 'Jordan Clark',
 'Adam Tomlinson',
 'Harry Taylor',
 'Brandon Starcevich',
 'Orazio Fantasia',
 'Connor Budarick',
 'Daniel Talia',
 'Phil Davis',
 'Benjamin Stratton',
 'James Frawley',
 'Fischer McAsey',
 'Ryan Gardner',
 'Michael Walters',
 'Kyle Langford',
 'Shai Bolton',
 'Isaac Smith',
 'Tom Phillips',
 'Sam Powell-Pepper',
 'Brett Bewley',
 'Justin Westhoff',
 'Jarrod Brander',
 'Nick Blakey',
 'Tom Scully',
 'Steven Motlop',
 'Ben King',
 'Paddy Dow',
 'Jonathon Patton',
 'Chayce Jones']